In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, date as dt_date  # Importing both datetime class and date class with an alias
from datetime import time, timedelta
import pytz
import os

In [2]:
file_name = 'naked_puts_results.csv'
file_name3 = 'firstFridayEnd.txt'

# File Reading with Error Handling
try:
    df = pd.read_csv(file_name)
except FileNotFoundError:
    print(f"Error: {file_name} not found.")
    exit(1)

try:
    with open(file_name3, 'r') as file:
        latestFridayDate = file.read().strip()
        print(latestFridayDate)
except FileNotFoundError:
    print(f"Error: {file_name3} not found.")
    exit(1)

2023-08-25


In [3]:
def is_market_closed(target_date):
    market_close_time = time(16, 0)
    
    # Define Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')
    
    # Check if target_date is already timezone-aware
    if target_date.tzinfo is None or target_date.tzinfo.utcoffset(target_date) is None:
        # Convert target_date to Eastern Time Zone
        target_date = eastern.localize(target_date)
    else:
        # Convert to Eastern Time if it's in a different timezone
        target_date = target_date.astimezone(eastern)
    
    # Get the current time in Eastern Time Zone
    current_time = datetime.now(eastern).time()

    # Check if the market has closed for the target date
    if target_date.date() < datetime.now(eastern).date() or \
       (target_date.date() == datetime.now(eastern).date() and current_time >= market_close_time):
        return True
    else:
        return False

In [4]:
def get_batch_ticker_prices(tickers, date):
    # Define Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')

    # Convert date to 'YYYY-MM-DD' format string if it's a datetime.date object
    if isinstance(date, dt_date):  # Using the alias dt_date here
        date = date.strftime('%Y-%m-%d')

    # Convert target_date to Eastern Time Zone
    target_date = eastern.localize(datetime.strptime(date, '%Y-%m-%d'))  # Using datetime class here

    if not is_market_closed(target_date):
        print("The market has not closed yet for this date.")
        return None
    
    # Convert the end date to the next day
    end_date = (target_date + timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        # Fetch data
        data = yf.download(tickers, start=date, end=end_date)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None
    
    if data.empty:
        print("No data found.")
        return None

    return data['Close']


### Sorting expired puts by expiration date

In [6]:
# Get the current date and time
today_datetime = datetime.now(pytz.timezone('US/Eastern'))

# Convert the date to YYYY-MM-DD format
todayStr = today_datetime.strftime("%Y-%m-%d")

# Convert todayStr to a datetime date object for more accurate comparison
today_date = pd.to_datetime(todayStr).date()

# Get the current time in HH:MM format
current_time = today_datetime.strftime("%H:%M")

# Print the date and time
print("Today's date:", todayStr)
print("Current Eastern Time:", current_time)

# Convert date strings to datetime objects for more reliable comparison
expiration_dates = pd.to_datetime(df['Expiration Date']).dt.date.unique()

# Create an empty list to store the expired dates
expiredOptionDates = []

# Check if today's date is greater than any of the dates in the expiration_dates
for date in expiration_dates:
    if today_date > date or (today_date == date and current_time >= '16:30'):
        expiredOptionDates.append(date)

if expiredOptionDates:
    for date in expiredOptionDates:
        date_str = date.strftime('%Y-%m-%d')
        currentDf = df[df['Expiration Date'] == date_str].copy()  # Make a copy if you intend to modify this slice

        if currentDf.empty:
            print(f"No data found for date {date_str}")
            continue

        unique_etfs = list(currentDf['ETF'].unique())
        tickerPrices = get_batch_ticker_prices(unique_etfs, date)

        if tickerPrices is None:
            print("Ticker prices could not be fetched.")
            continue

        # Convert strike column to float for numerical comparison
        currentDf.loc[:, 'strike'] = currentDf['strike'].astype(float)

        # Go through each row in currentDf
        for index, row in currentDf.iterrows():
            etf_price = tickerPrices.get(row['ETF'], None)
            
            if etf_price is None:
                print(f"No price information found for ETF {row['ETF']}.")
                continue

            if isinstance(etf_price, pd.Series):
                etf_price = etf_price.iloc[0]

            currentDf.loc[index, 'Stock Price'] = etf_price

            if row['strike'] < etf_price:
                currentDf.loc[index, 'Status'] = 'Success'
            else:
                currentDf.loc[index, 'Status'] = 'Failed'

        csv_file_path = 'Data/putsDataSuccessFailed.csv'
         # Check if the naked_puts_results.csv file exists and if it's not empty
        if os.path.exists(csv_file_path) and os.path.getsize(csv_file_path) > 0:
            
            # Read the naked_puts_results.csv file into a DataFrame
            file_df = pd.read_csv(csv_file_path)
            
            # Find common contractSymbols
            common_symbols = set(file_df['contractSymbol']).intersection(set(currentDf['contractSymbol']))

            for symbol in common_symbols:
                # Replace all column values for rows with this contractSymbol
                currentDf.loc[currentDf['contractSymbol'] == symbol] = file_df.loc[file_df['contractSymbol'] == symbol].values
                
            # Append new rows
            desired_columns = ['contractSymbol','Expiration Date', 'ETF', 'Stock Price', 'lastPrice', 'bid', 'ask', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'Original ROI (%)', 'highPrice', 'Status']
            new_rows = file_df.loc[~file_df['contractSymbol'].isin(currentDf['contractSymbol']), desired_columns]
            currentDf = pd.concat([currentDf, new_rows], ignore_index=True)
            
        # Save the updated results DataFrame back to the naked_puts_results.csv file
        currentDf.to_csv(csv_file_path, index=False)

# Change the dates in expiredOptionDates to strings using .strftime('%Y-%m-%d')
expiredOptionDates = [date.strftime('%Y-%m-%d') for date in expiredOptionDates]

# Remove all the expiredOptionDates in df
df = df[~df['Expiration Date'].isin(expiredOptionDates)]

# Save the updated results DataFrame back to the naked_puts_results.csv file
df.to_csv(file_name, index=False)

Today's date: 2023-08-25
Current Eastern Time: 18:46
[*********************100%%**********************]  40 of 40 completed
